In [52]:
%load_ext autoreload
%autoreload 2

%pip install -r requirements.txt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Note: you may need to restart the kernel to use updated packages.


In [53]:
import sys
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision
import torchvision.models as models
from torchvision import transforms
from datasets import load_dataset, concatenate_datasets

In [54]:
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Check for CUDA support
print(f"Is CUDA available? {torch.cuda.is_available()}")

# Set the device
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using device: {device}")


PyTorch version: 2.1.0
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Is CUDA available? False
Using device: mps


In [55]:
from EarlyExitModel import EarlyExitModel

def create_model_with_exits(model_type, num_classes):
    if model_type == "resnet":
        resnet = models.resnet50(pretrained=True)
        resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)
        
        model = EarlyExitModel(resnet, num_classes, device)
        model.clear_exits()
        exit_layers = [model.add_exit(layer) for layer in ('layer1', 'layer2', 'layer3')]
        
    elif model_type == "vgg":
        vgg = models.vgg16(pretrained=True)
        vgg.classifier[-1] = nn.Linear(vgg.classifier[-1].in_features, num_classes)
        
        model = EarlyExitModel(vgg, num_classes, device)
        model.clear_exits()
        exit_layers = [model.add_exit(layer, model_type) for layer in ('10', '21', '28')]
    else:
        raise ValueError(f"Unknown model type: {model_type}")
    
    model.to(device)
    return model
        
        

In [56]:
# choose the model
model_type = "vgg" # "resnet" or "vgg"

model = create_model_with_exits(model_type, num_classes=10)

model

EarlyExitModel(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): OptionalExitModule(
        (module): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), paddin

In [57]:
from DataLoader import CustomDataset

if model_type == "resnet":
    # use the imagenette dataset
    hf_dataset = load_dataset("frgfm/imagenette", '320px')
    hf_dataset = concatenate_datasets(hf_dataset.values())
elif model_type == "vgg":
    # use the cifar10 dataset
    hf_dataset = load_dataset("cifar10")
    hf_dataset = concatenate_datasets(hf_dataset.values())
    

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


torch_dataset = CustomDataset(hf_dataset, transform=transform)

batch_size = 32 if model_type == "resnet" else 64

test_size = 0.2
test_volume = int(test_size * len(torch_dataset))
train_volume = len(torch_dataset) - test_volume

train_dataset, test_dataset = random_split(torch_dataset, [train_volume, test_volume])
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=False, 
    num_workers=4
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4
)

## Early Exit Model Training

If you want tensorboard support, you need to run the following commands on **macOS**:

This clears your logs:
```
rm -rf runs/*
```

This shows your public IP address:
```
dig -4 TXT +short o-o.myaddr.l.google.com @ns1.google.com
```

This starts tensorboard at this address:
```
tensorboard --host 0.0.0.0 --logdir=runs
```



If you want tensorboard support, you need to run the following commands on **Windows**:

This clears your logs:
```
rmdir /s /q runs
```

This shows your IP address:
```
powershell -command "$ipAddress = (Invoke-WebRequest -Uri 'http://ipinfo.io/ip').Content.Trim(); Write-Host 'Your IP address is: ' $ipAddress"  
```

This starts tensorboard at this address:
```
tensorboard --host=0.0.0.0 --logdir=runs
```

In [58]:
from EarlyExitTrainer import ModelTrainer

trainer = ModelTrainer(model, device)

In [59]:
# train the classifiers
trainer.train_classifiers(train_dataloader, epoch_count=30, validation_loader=test_dataloader)


Training classifier for exit 1


Epoch 0:   0%|                                                             | 0/1500 [00:00<?, ?it/s]

Epoch 0:  22%|█████▎                  | 335/1500 [03:00<10:31,  1.85it/s, Loss=1.88, Accuracy=0.406]

In [ ]:
# train the exits

trainer.train_exit_layers(train_dataloader, epoch_count=30, validation_loader=test_dataloader)

Epoch 0:   0%|                                                              | 0/335 [00:00<?, ?it/s]

gate_summation:  tensor(111.6650, device='mps:0', grad_fn=<AsStridedBackward0>) exit_costs:  tensor(15.0685, device='mps:0', grad_fn=<AsStridedBackward0>)


Epoch 1:   0%|                                                              | 0/335 [00:00<?, ?it/s]

gate_summation:  tensor(2.7934, device='mps:0', grad_fn=<AsStridedBackward0>) exit_costs:  tensor(31.5549, device='mps:0', grad_fn=<AsStridedBackward0>)


Epoch 2:   0%|                                                              | 0/335 [00:00<?, ?it/s]

gate_summation:  tensor(1.1086, device='mps:0', grad_fn=<AsStridedBackward0>) exit_costs:  tensor(31.9774, device='mps:0', grad_fn=<AsStridedBackward0>)


Epoch 3:   0%|                                                              | 0/335 [00:00<?, ?it/s]